In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# 1. 클러스터링에 사용할 핵심 변수 선택
# (Churn_Score와 Slope_Spend 두 가지로 군집화)
cluster_cols = ['Churn_Score', 'Slope_Spend']

# 데이터 준비
X_cluster = df_final[cluster_cols].copy()

# 2. 스케일링 (필수)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_cluster)

# 3. K-Means 클러스터링 (K=3)
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(X_scaled)

# 원본 데이터에 클러스터 라벨 붙이기
df_final['Cluster_ID'] = clusters

# -------------------------------------------------------
# 4. 결과 해석 (통계)
# -------------------------------------------------------
print("\n[클러스터별 평균 특성 확인]")
print(df_final.groupby('Cluster_ID')[cluster_cols].mean())

# -------------------------------------------------------
# 5. 시각화 (수정됨: 클러스터링 변수와 축 일치시키기)
# -------------------------------------------------------
plt.figure(figsize=(10, 6))

# X축: 소비 기울기 (Slope_Spend)
# Y축: 이탈 점수 (Churn_Score)
sns.scatterplot(
    data=df_final, 
    x='Slope_Spend', 
    y='Churn_Score', 
    hue='Cluster_ID', 
    palette='viridis',
    alpha=0.6
)

plt.title('Clustering Result: Slope vs Score')
plt.xlabel('Slope Spend (소비 기울기)')
plt.ylabel('Churn Score (이탈 점수)')

# 기준선 추가 (보기 편하게)
plt.axvline(x=0, color='r', linestyle='--', label='Slope=0') 
plt.legend()
plt.show()

# -------------------------------------------------------
# 6. 클러스터 0 (또는 특정 그룹) 비중 확인 코드
# -------------------------------------------------------
# 전체 인원수 및 비율 계산
cluster_counts = df_final['Cluster_ID'].value_counts().sort_index()
cluster_ratios = df_final['Cluster_ID'].value_counts(normalize=True).sort_index() * 100

print("-" * 30)
print("[클러스터별 분포 현황]")
df_summary = pd.DataFrame({
    '인원수(명)': cluster_counts,
    '비율(%)': cluster_ratios.round(2)
})
print(df_summary)
print("-" * 30)

# 만약 0번 그룹을 위험군으로 본다면:
target_group = 0 
print(f"▶ [Cluster {target_group}] 위험군 비율: {cluster_ratios[target_group]:.2f}%")
print(f"▶ [Cluster {target_group}] 인원수: {cluster_counts[target_group]:,}명")